In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

C:\Users\Nayan\AppData\Local\Temp\ipykernel_19460\1399112611.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\Nayan\AppData\Local\Temp\ipykernel_19460\1399112611.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [52]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')
spacex_df.to_csv('spacex_launch_dash.csv')
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
spacex_df.head(2)

,Unnamed: 0.1,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0


In [70]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                html.Div([
                                    html.Label('Select Launch Site:'),
                                    dcc.Dropdown(
                                        id='site-dropdown',
                                        options=[
                                                {'label':'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                                {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'},
                                                {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
                                                {'label':'VAFB SLC-4E', 'value':'VAFB SLC-4E'}
                                                ],
                                        value='ALL',
                                        placeholder='Select a Launch Site here',
                                        searchable=True
                                        ),
                                ]),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.Div([
                                    html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                    dcc.RangeSlider(id='payload-slider',
                                               min=0, max=10000, step=1000,
                                               
                                               value=[min_payload, max_payload]),
                                ]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

#Defining a function to get pie chart as a return

def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', names='Launch Site',
                    title='Success Count of all Launch Sites')
        return fig
    else:
        filtered_df=spacex_df[spacex_df['Launch Site']==entered_site]
        filtered_df=filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig = px.pie(filtered_df, values='class count', names=['Failure','Successful'],
                     color_discrete_sequence=['Red','Green'],
                    title='Total Success Launches for {} Launch Site'.format(entered_site))
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
             [Input(component_id='site-dropdown', component_property='value'),
             Input(component_id='payload-slider', component_property='value')])

#Defining a function to get a scatter plot as a return

def get_scatter_plot(entered_site, selected_payload_range):
    scatter_df = spacex_df[spacex_df['Payload Mass (kg)'].between(selected_payload_range[0], selected_payload_range[1])]
    if entered_site == 'ALL':
        fig = px.scatter(scatter_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         hover_name='Launch Site'
                         )
        return fig
    else:
        scatter_range_df = scatter_df[scatter_df['Launch Site']==entered_site]
        fig = px.scatter(scatter_range_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         hover_name='Launch Site'
                        )
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()